In [1]:
!which python

/home/kamal/Downloads/onboarding/scibert-procodex/.venv/bin/python


In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
with open("../data/raw/20230821_full_data.pkl", "rb") as handle:
    data = pickle.load(handle)

In [4]:
data

,id,title,keywords,content,target
0,7OT5chJU5Y1L6A,"Electromagnetic Compatibility Regulations, 2016",Electromagnetic compatibility;EMC;Radio freque...,VERŻJONI ELETTRONIKA\nB\n\n1277\n\nB 1277\nSup...,Relevant
1,QXU5clgY5Y1L6A,Regulation governing the implementation of cha...,Construction of buses,"Netherlands: RV 2009 Chap. 3, 5, 6;\nApp. IIIA...",Relevant
2,SsT5cxDw5Y1L6A,Traffic Implementation Regulations - Electrom...,Electromagnetic compatibility;EMC;Radio freque...,Österreich: KDV § 17b\n\nElektromagnetische Ve...,Relevant
3,tLf5surH5Y1L6A,NHTSA Federal Automated Vehicles Policy (FAVP)...,Autonomous driving,AUTOMATED DRIVING SYSTEMS\nA Vision for Safety...,Relevant
4,KxZ5sjo75Y1L6A,Decree of the Minister of Equipment and Transp...,Flammability of materials,Morocco: Arr. 2915-10\n\nFlammability of mater...,Relevant
...,...,...,...,...,...
2376,7wpFaHSu5Y1L6A,Luxembourg: New European Standards Applicable ...,Aerospace Products;Building / Construction Pro...,JOURNAL OFFICIEL\nDU GRAND-DUCHÉ DE LUXEMBOURG...,Not relevant
2377,8QkF$3ly5Y1L6A,Luxembourg: New European Standards Applicable ...,Declaring/certifying conformity;Radio/Wireless...,JOURNAL OFFICIEL\nDU GRAND-DUCHÉ DE LUXEMBOURG...,Not relevant
2378,zGpNEAGB5Y1L6A,Lithuania: Implementation of Directive (EU) 20...,Accessibility;Computers;Consumer Protection;De...,LIETUVOS RESPUBLIKOS\nGAMINIŲ IR PASLAUGŲ PRIE...,Not relevant
2379,K2jJ$ks45Y1L6A,"USA: Event Data Recorders for Vehicles, Rules,...",Automotive;Retention/Destruction of records,"Federal Register / Vol. 87, No. 119 / Wednesda...",Not relevant


In [38]:
import os
import re
import string
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

import warnings 
warnings.filterwarnings('ignore')

# 1. lower case
def lower_case(text):
    return text.lower()

# 2. remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# 3. remove stop words
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_text)

# 4. remove numbers
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# 5. remove short words
def remove_short_words(text):
    word_tokens = word_tokenize(text)
    filtered_text = [w for w in word_tokens if len(w) > 2]
    return ' '.join(filtered_text)

# 6. lemmatize
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(w) for w in word_tokens]
    return ' '.join(lemmatized_text)

# 8. remove non-ascii characters
def remove_non_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

# 9. remove extra spaces
def remove_extra_spaces(text):
    return re.sub(' +', ' ', text)

# 16. remove extra non-breaking spaces
def remove_extra_non_breaking_spaces(text):
    text = re.sub('\xa0+', '', text)
    text = re.sub('\x0c', '', text)

    return text

def remove_dashes_underscores(text):
    text = text.replace("-","")
    text=text.replace("_","")

    return text

def remove_special_characters(text):
    # Define a regular expression pattern to match special characters
    pattern = r'[^a-zA-Z0-9\s]'  # This pattern matches anything that's not alphanumeric or whitespace

    # Use re.sub() to replace matched patterns with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text
    
def remove_urls(text):
    # Define a regular expression pattern to match URLs
    pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Use re.sub() to replace matched URLs with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

def filter_text(text):
    # If 90% of the characters are space or numbers, remove the row?
    count = 0

    for char in text:
        if char.isdigit() or char==" ":
            count+=1
    non_chars = count/len(text)

    return non_chars

In [39]:
def process_target(label):
    label_map={
            "Relevant":1,
            "Not relevant":0
    }
    return label_map[label]

def preprocess(text):
    pass

In [40]:
test = pd.read_excel("../data/raw/test_ids.xlsx")
test_ids = list(test["id"].values)

train = data[~data["id"].isin(test_ids)]
test = data[data["id"].isin(test_ids)]

In [44]:
x = train["content"][180].replace("\n", " ")
x = remove_urls(x)
x = remove_non_ascii(x)
x = remove_extra_non_breaking_spaces(x)
x = remove_dashes_underscores(x)
x = remove_special_characters(x)
x = remove_extra_spaces(x)
# x = filter_text(x)
x

'3142017 eCFR Code of Federal Regulations ELECTRONIC CODE OF FEDERAL REGULATIONS eCFR data is current as of March 10 2017 Amendment 49 CFRPART 571 View Printed Federal Register page 81 FR 90514 in PDF format Amendments published December 14 2016 in 81 FR 90514 EFFECTIVE DATES February 13 2017 3 Section 571141 is added to read as follows 571141 Standard No 141 Minimum Sound Requirements for Hybrid and Electric Vehicles S1 Scope This standard establishes performance requirements for pedestrian alert sounds for motor vehicles S2 Purpose The purpose of this standard is to reduce the number of injuries that result from electric and hybrid vehicle crashes with pedestrians by providing a sound level and sound characteristics necessary for these vehicles to be detected and recognized by pedestrians S3 Application This standard applies to a Electric vehicles with a gross vehicle weight rating GVWR of 4536 Kg or less that are passenger cars multipurpose passenger vehicles trucks or buses b Hybri